### Sprawozdanie - Generatory Liczb Losowych
<div style="text-align: right"> Wojciech Kosztyła </div>


In [1]:
import random
import numpy as np
import random
import scipy as cp
from math import *
import matplotlib.pyplot as plt

#### Zadanie 1 - Testowanie generatorów liczb losowych

Dla obydwu generatorów liczb losowych (Mersenne Twister oraz PCG64) oraz dla $n=10,1000,5000$ wylosuj $n$ liczb losowych pochodzących z rozkładu jednostajnego i wykonaj następujące kroki.
1. Zwizualizuj na wykresie rozkład liczb w 10 równych przedziałach.
2. Sprawdź, dla ilu liczb spełniona jest nierówność $x_{i} < x_{i+1}$. Ile powinno ich być dla idealnego generatora?
3. Zaimplementuj jeden z testów zdefiniowanych w rozdziale drugim artykułu [(link)](https://csrc.nist.gov/publications/detail/sp/800-22/rev-1a/final) i wykorzystaj go do sprawdzenia wylosowanego ciągu liczb.

Czy widać różnice pomiędzy generatorami? Czy wraz z rosnącym $n$ coś się zmienia?

Zaimplementowalem tworzenie histogramu na te dwa sposoby.

Wybrany przeze mnie test, to "Frequency (Monobit) Test", przy czym korzystam z przybliżenia wylosowanej liczby z zakresu [0,1) do bita.

In [2]:
def histogram_MarsenneTwister(n, ilosc_przedzialow):
    histogram = []
    for _ in range(ilosc_przedzialow):
        histogram.append(0)

    for _ in range(n):
        wylosowana_wartosc = random.uniform(0,1)

        for i in range(ilosc_przedzialow):
            if wylosowana_wartosc >= i/ilosc_przedzialow and wylosowana_wartosc < (i+1)/ilosc_przedzialow:
                histogram[i] += 1
                break

    return histogram

def histogram_MCG64(n, ilosc_przedzialow):
    histogram = []
    for _ in range(ilosc_przedzialow):
        histogram.append(0)

    for _ in range(n):
        wylosowana_wartosc = np.random.uniform()

        for i in range(ilosc_przedzialow):
            if wylosowana_wartosc >= i/ilosc_przedzialow and wylosowana_wartosc < (i+1)/ilosc_przedzialow:
                histogram[i] += 1
                break

    return histogram

print(" MarsenneTwister:\n Wartości poszczególnych 'kolumn': {} \n".format(histogram_MarsenneTwister(5000, 10)))
print(" MCG64:\n Wartości poszczególnych 'kolumn': {} \n".format(histogram_MCG64(5000, 10)))

 MarsenneTwister:
 Wartości poszczególnych 'kolumn': [481, 494, 495, 480, 513, 449, 532, 519, 499, 538] 

 MCG64:
 Wartości poszczególnych 'kolumn': [544, 470, 530, 474, 544, 498, 500, 479, 505, 456] 



Ta wersja implementacji wykonywała pierwszą, a zarazem najistotniejszą część zadania - generowała histogram (jeszcze nie wizualnie) z losowych wartości.

Jesteśmy też w stanie stwierdzić, czy jest to rzeczywiście rozkład jednostajny - na to wygląda, gdyż nie ma żadnych "rzucających się w oczy" "górek".

Następnie rozszerzyłem te implementacje o podpunkty 2.(zliczenie wystąpień nierówności) i 3.(implementacja jednego z testów).

In [3]:
def histogram_MarsenneTwister(n, ilosc_przedzialow):
    histogram = []
    for _ in range(ilosc_przedzialow):
        histogram.append(0)

    licznik_warunku = 0     # do podpunktu 2
    ostatnia_wartosc = 1000

    temp_suma = 0

    for _ in range(n):
        wylosowana_wartosc = random.uniform(0,1)
        if wylosowana_wartosc > ostatnia_wartosc:
            licznik_warunku+=1

        if wylosowana_wartosc>=0.5:
            temp_suma+=1
        else:
            temp_suma-=1

        for i in range(ilosc_przedzialow):
            if wylosowana_wartosc >= i/ilosc_przedzialow and wylosowana_wartosc < (i+1)/ilosc_przedzialow:
                histogram[i] += 1
                break
        ostatnia_wartosc = wylosowana_wartosc

    return histogram, licznik_warunku, erfc(abs(temp_suma)/sqrt(2*n))

def histogram_MCG64(n, ilosc_przedzialow):
    histogram = []
    for _ in range(ilosc_przedzialow):
        histogram.append(0)

    licznik_warunku = 0
    ostatnia_wartosc = 1000

    temp_suma = 0

    for _ in range(n):
        wylosowana_wartosc = np.random.uniform()
        if wylosowana_wartosc > ostatnia_wartosc:
            licznik_warunku+=1

        if wylosowana_wartosc>=0.5:
            temp_suma+=1
        else:
            temp_suma-=1

        for i in range(ilosc_przedzialow):
            if wylosowana_wartosc >= i/ilosc_przedzialow and wylosowana_wartosc < (i+1)/ilosc_przedzialow:
                histogram[i] += 1
                break
        ostatnia_wartosc = wylosowana_wartosc

    return histogram, licznik_warunku, erfc(abs(temp_suma)/sqrt(2*n))


temp_histogram_MarsenneTwister, l, erfc_MarsenneTwister = histogram_MarsenneTwister(5000, 10)
print(" MarsenneTwister:\n Wartości poszczególnych 'kolumn': {} \n Ilosc prawd warunku 2) :{} \t Wynik testu (jeśli >=0.01 to zbiór jest losowy): {}\n".format(temp_histogram_MarsenneTwister, l, erfc_MarsenneTwister))

temp_histogram_MCG64, l, erfc_MCG64 = histogram_MCG64(5000, 10)
print(" MCG64:\n Wartości poszczególnych 'kolumn': {} \n Ilosc prawd warunku 2) :{} \t Wynik testu (jeśli >=0.01 to zbiór jest losowy): {}\n".format(temp_histogram_MCG64, l, erfc_MCG64))

 MarsenneTwister:
 Wartości poszczególnych 'kolumn': [503, 549, 477, 501, 511, 500, 499, 506, 457, 497] 
 Ilosc prawd warunku 2) :2495 	 Wynik testu (jeśli >=0.01 to zbiór jest losowy): 0.2461892491250375

 MCG64:
 Wartości poszczególnych 'kolumn': [500, 491, 541, 520, 490, 511, 460, 500, 469, 518] 
 Ilosc prawd warunku 2) :2504 	 Wynik testu (jeśli >=0.01 to zbiór jest losowy): 0.23485728854500548



Co do podpunktu 2. - nierówność $x_{i} < x_{i+1}$ spełniana jest przez połowę z wylosowanych liczb.

Uzasadnienie można szukać w statystyce - skoro jest to rozkład jednostajny, to prawdopodobieństwo wylosowania po liczbie $x_{i}$ liczby większej jest równe $1 - x_{i}$, a wartość oczekiwana to $0.5$.
Jest to zależność liniowa, więc można prosto przekształcić $average(1 - x_{i}) = 1 - wartość oczekiwana = 1 - 0.5 = 0.5$

"Rzucamy $n$ razy monetą i zliczamy wypadnięcia reszki".

Co do podpunktu 3. - Test sprawdza proporcję zer do jedynek (a w tym przypadku liczb z przedziału $[0,0.5)$ i $[0.5,1)$).

Taka zamiana jest jak najbardziej poprawna, gdyż poprawny generator liczb losowych powinien wylosować taką samą ilość liczb w dwóch przedziałach równej długości.

Wartość "*erfc(abs(temp_suma)/sqrt(2n))*" to tzw. "*P-value*".
Gdyby była ona mała, świadczyłoby to, że "*temp_suma*" wyszła duża, a ta z kolei powinna oscylować blisko 0.